In [1]:
# Some global configuration for this workbook.
reset()
%display latex
Partitions.options.latex="list"

In [2]:
# The heart of the combinatorics of DT theory is 3d partitions.
load("partition_3d.sage")

PPs = Partitions3d([2], [1], [3,1])
PPs

3d partitions with legs [2], [1], [3, 1]

In [3]:
PPs.random_element_with_num_boxes(25).plot()

Graphics3d Object

In [4]:
[len(list(PPs.with_num_boxes(k))) for k in range(7)]

[1, 4, 14, 39, 102, 245, 562]

In [5]:
# Load everything related to computing the (bare) vertex
# in either Donaldson-Thomas or Pandharipande-Thomas theory.
load("bare_vertex.sage")

# This package includes some default setup for variables.
# We'll use the default setup for now.
x, y, z = Default.x, Default.y, Default.z
q = Default.q

In [6]:
###########################################################
# Let's start with the vertex with all empty legs.
# ct = cohomology theory: either KTheory (default if unspecified) 
#                         or Cohomology.
V000 = BareVertex([], [], [], ct=KTheory) 
V000

Bare vertex with legs ([], [], []) using K-theory

In [7]:
# Compute the vertex in K-theory, up to O(q^3)
v000 = V000.series_unreduced(3)
v000

1 + ((x^4*y^4*z^4 - x^4*y^2*z^2 - x^2*y^4*z^2 - x^2*y^2*z^4 + x^2*y^2 + x^2*z^2 + y^2*z^2 - 1)/(-x^3*y^3*z^3 + x^3*y^3*z + x^3*y*z^3 + x*y^3*z^3 - x^3*y*z - x*y^3*z - x*y*z^3 + x*y*z))*q + ((x^10*y^10*z^10 - x^10*y^10*z^8 - x^10*y^8*z^10 - x^8*y^10*z^10 - x^10*y^10*z^6 - x^10*y^6*z^10 - x^6*y^10*z^10 + x^10*y^8*z^6 + x^8*y^10*z^6 + x^10*y^6*z^8 + 3*x^8*y^8*z^8 + x^6*y^10*z^8 + x^8*y^6*z^10 + x^6*y^8*z^10 + x^10*y^8*z^4 + x^8*y^10*z^4 + x^10*y^6*z^6 + 2*x^8*y^8*z^6 + x^6*y^10*z^6 + x^10*y^4*z^8 + 2*x^8*y^6*z^8 + 2*x^6*y^8*z^8 + x^4*y^10*z^8 + x^8*y^4*z^10 + x^6*y^6*z^10 + x^4*y^8*z^10 - 3*x^8*y^8*z^4 - 4*x^8*y^6*z^6 - 4*x^6*y^8*z^6 - 3*x^8*y^4*z^8 - 4*x^6*y^6*z^8 - 3*x^4*y^8*z^8 - x^8*y^8*z^2 - 2*x^10*y^4*z^4 - 4*x^8*y^6*z^4 - 4*x^6*y^8*z^4 - 2*x^4*y^10*z^4 - 4*x^8*y^4*z^6 - 4*x^6*y^6*z^6 - 4*x^4*y^8*z^6 - x^8*y^2*z^8 - 4*x^6*y^4*z^8 - 4*x^4*y^6*z^8 - x^2*y^8*z^8 - 2*x^4*y^4*z^10 + 2*x^8*y^6*z^2 + 2*x^6*y^8*z^2 + 3*x^8*y^4*z^4 + 6*x^6*y^6*z^4 + 3*x^4*y^8*z^4 + 2*x^8*y^2*z^6 + 6*x^6*y^4*z^6 + 6*x^4*y^6*z^6 + 2*x^2*y^8*z^6 + 2*x^6*y^2*z^8 + 3*x^4*y^4*z^8 + 2*x^2*y^6*z^8 + 2*x^8*y^4*z^2 + 3*x^6*y^6*z^2 + 2*x^4*y^8*z^2 + 2*x^8*y^2*z^4 + 6*x^6*y^4*z^4 + 6*x^4*y^6*z^4 + 2*x^2*y^8*z^4 + 3*x^6*y^2*z^6 + 6*x^4*y^4*z^6 + 3*x^2*y^6*z^6 + 2*x^4*y^2*z^8 + 2*x^2*y^4*z^8 - 2*x^6*y^6 - x^8*y^2*z^2 - 4*x^6*y^4*z^2 - 4*x^4*y^6*z^2 - x^2*y^8*z^2 - 4*x^6*y^2*z^4 - 4*x^4*y^4*z^4 - 4*x^2*y^6*z^4 - 2*x^6*z^6 - 4*x^4*y^2*z^6 - 4*x^2*y^4*z^6 - 2*y^6*z^6 - x^2*y^2*z^8 - 3*x^6*y^2*z^2 - 4*x^4*y^4*z^2 - 3*x^2*y^6*z^2 - 4*x^4*y^2*z^4 - 4*x^2*y^4*z^4 - 3*x^2*y^2*z^6 + x^6*y^2 + x^4*y^4 + x^2*y^6 + x^6*z^2 + 2*x^4*y^2*z^2 + 2*x^2*y^4*z^2 + y^6*z^2 + x^4*z^4 + 2*x^2*y^2*z^4 + y^4*z^4 + x^2*z^6 + y^2*z^6 + x^4*y^2 + x^2*y^4 + x^4*z^2 + 3*x^2*y^2*z^2 + y^4*z^2 + x^2*z^4 + y^2*z^4 - x^4 - y^4 - z^4 - x^2 - y^2 - z^2 + 1)/(-x^8*y^8*z^8 + x^8*y^8*z^6 + x^8*y^6*z^8 + x^6*y^8*z^8 + x^8*y^8*z^4 - x^8*y^6*z^6 - x^6*y^8*z^6 + x^8*y^4*z^8 - x^6*y^6*z^8 + x^4*y^8*z^8 - x^8*y^8*z^2 - x^8*y^6*z^4 - x^6*y^8*z^4 - x^8*y^4*z^6 + x^6*y^6*z^6 - x^4*y^8*z^6 - x^8*y^2*z^8 - x^6*y^4*z^8 - x^4*y^6*z^8 - x^2*y^8*z^8 + x^8*y^6*z^2 + x^6*y^8*z^2 - x^8*y^4*z^4 + x^6*y^6*z^4 - x^4*y^8*z^4 + x^8*y^2*z^6 + x^6*y^4*z^6 + x^4*y^6*z^6 + x^2*y^8*z^6 + x^6*y^2*z^8 - x^4*y^4*z^8 + x^2*y^6*z^8 + x^8*y^4*z^2 - x^6*y^6*z^2 + x^4*y^8*z^2 + x^8*y^2*z^4 + x^6*y^4*z^4 + x^4*y^6*z^4 + x^2*y^8*z^4 - x^6*y^2*z^6 + x^4*y^4*z^6 - x^2*y^6*z^6 + x^4*y^2*z^8 + x^2*y^4*z^8 - x^8*y^2*z^2 - x^6*y^4*z^2 - x^4*y^6*z^2 - x^2*y^8*z^2 - x^6*y^2*z^4 + x^4*y^4*z^4 - x^2*y^6*z^4 - x^4*y^2*z^6 - x^2*y^4*z^6 - x^2*y^2*z^8 + x^6*y^2*z^2 - x^4*y^4*z^2 + x^2*y^6*z^2 - x^4*y^2*z^4 - x^2*y^4*z^4 + x^2*y^2*z^6 + x^4*y^2*z^2 + x^2*y^4*z^2 + x^2*y^2*z^4 - x^2*y^2*z^2))*q^2 + O(q^3)

In [8]:
# Package includes a utility function for pretty-printing
pretty(v000.coefficients()[1])

-(x*y + 1)*(x*y - 1)*(x*z + 1)*(x*z - 1)*(y*z + 1)*(y*z - 1)/((x + 1)*(x - 1)*x*(y + 1)*(y - 1)*y*(z + 1)*(z - 1)*z)

In [9]:
# To avoid taking square roots of variables living in a (Laurent) 
# polynomial ring (which is an operation Sage does not allow),
# we automatically square all variables upon applying the K-theoretic 
# a-hat genus \hat{a}(x) = (sqrt(x) - 1/sqrt(x)).

# We view \hat{a} as a sort of measure on 3d partitions, hence the name:
v000.coefficients()[1] == -KTheory.measure(x + y + z - x*y - x*z - y*z)

True

In [10]:
###############################################################
# Let's check the (known) formula for the K-theoretic (no legs) vertex.
# This requires us to implement the plethystic exponential.

def adams(k):
    wR = Default.weight_ring
    qR = Default.boxcounting_ring.power_series_ring() # to avoid a bug
    adams_wR = Hom(wR, wR)([x^k, y^k, z^k])
    adams_qR = Hom(qR, qR)([q^k])
    return adams_qR * Hom(qR, qR)(adams_wR) # lift adams_wR to Hom(qR, qR)

def plethystic_exponential(f, qprec):
    g = sum(adams(k)(f)/k for k in range(1, qprec)).add_bigoh(qprec)
    return sum(g^k / factorial(k) for k in range(qprec))

In [11]:
adams(4)

Composite map:
  From: Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Integer Ring
  To:   Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Integer Ring
  Defn:   Ring endomorphism of Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Integer Ring
          Defn: Induced from base ring by
                Ring endomorphism of Fraction Field of Multivariate Polynomial Ring in x, y, z over Integer Ring
                  Defn: x |--> x^4
                        y |--> y^4
                        z |--> z^4
        then
          Ring endomorphism of Power Series Ring in q over Fraction Field of Multivariate Polynomial Ring in x, y, z over Integer Ring
          Defn: q |--> q^4

In [12]:
# An example: this yields the MacMahon function \prod (1-q^n)^-n
plethystic_exponential(q/(1-q)^2, qprec=10)

1 + q + 3*q^2 + 6*q^3 + 13*q^4 + 24*q^5 + 48*q^6 + 86*q^7 + 160*q^8 + 282*q^9 + O(q^10)

In [13]:
# Check the formula for the K-theoretic vertex
kappa = x*y*z
plethystic_exponential(-q/(1-q*kappa)/(1-q/kappa) * 
                       KTheory.measure(x+y+z - x*y-x*z-y*z), qprec=3)

1 + ((-x^4*y^4*z^4 + x^4*y^2*z^2 + x^2*y^4*z^2 + x^2*y^2*z^4 - x^2*y^2 - x^2*z^2 - y^2*z^2 + 1)/(x^3*y^3*z^3 - x^3*y^3*z - x^3*y*z^3 - x*y^3*z^3 + x^3*y*z + x*y^3*z + x*y*z^3 - x*y*z))*q + ((-x^10*y^10*z^10 + x^10*y^10*z^8 + x^10*y^8*z^10 + x^8*y^10*z^10 + x^10*y^10*z^6 + x^10*y^6*z^10 + x^6*y^10*z^10 - x^10*y^8*z^6 - x^8*y^10*z^6 - x^10*y^6*z^8 - 3*x^8*y^8*z^8 - x^6*y^10*z^8 - x^8*y^6*z^10 - x^6*y^8*z^10 - x^10*y^8*z^4 - x^8*y^10*z^4 - x^10*y^6*z^6 - 2*x^8*y^8*z^6 - x^6*y^10*z^6 - x^10*y^4*z^8 - 2*x^8*y^6*z^8 - 2*x^6*y^8*z^8 - x^4*y^10*z^8 - x^8*y^4*z^10 - x^6*y^6*z^10 - x^4*y^8*z^10 + 3*x^8*y^8*z^4 + 4*x^8*y^6*z^6 + 4*x^6*y^8*z^6 + 3*x^8*y^4*z^8 + 4*x^6*y^6*z^8 + 3*x^4*y^8*z^8 + x^8*y^8*z^2 + 2*x^10*y^4*z^4 + 4*x^8*y^6*z^4 + 4*x^6*y^8*z^4 + 2*x^4*y^10*z^4 + 4*x^8*y^4*z^6 + 4*x^6*y^6*z^6 + 4*x^4*y^8*z^6 + x^8*y^2*z^8 + 4*x^6*y^4*z^8 + 4*x^4*y^6*z^8 + x^2*y^8*z^8 + 2*x^4*y^4*z^10 - 2*x^8*y^6*z^2 - 2*x^6*y^8*z^2 - 3*x^8*y^4*z^4 - 6*x^6*y^6*z^4 - 3*x^4*y^8*z^4 - 2*x^8*y^2*z^6 - 6*x^6*y^4*z^6 - 6*x^4*y^6*z^6 - 2*x^2*y^8*z^6 - 2*x^6*y^2*z^8 - 3*x^4*y^4*z^8 - 2*x^2*y^6*z^8 - 2*x^8*y^4*z^2 - 3*x^6*y^6*z^2 - 2*x^4*y^8*z^2 - 2*x^8*y^2*z^4 - 6*x^6*y^4*z^4 - 6*x^4*y^6*z^4 - 2*x^2*y^8*z^4 - 3*x^6*y^2*z^6 - 6*x^4*y^4*z^6 - 3*x^2*y^6*z^6 - 2*x^4*y^2*z^8 - 2*x^2*y^4*z^8 + 2*x^6*y^6 + x^8*y^2*z^2 + 4*x^6*y^4*z^2 + 4*x^4*y^6*z^2 + x^2*y^8*z^2 + 4*x^6*y^2*z^4 + 4*x^4*y^4*z^4 + 4*x^2*y^6*z^4 + 2*x^6*z^6 + 4*x^4*y^2*z^6 + 4*x^2*y^4*z^6 + 2*y^6*z^6 + x^2*y^2*z^8 + 3*x^6*y^2*z^2 + 4*x^4*y^4*z^2 + 3*x^2*y^6*z^2 + 4*x^4*y^2*z^4 + 4*x^2*y^4*z^4 + 3*x^2*y^2*z^6 - x^6*y^2 - x^4*y^4 - x^2*y^6 - x^6*z^2 - 2*x^4*y^2*z^2 - 2*x^2*y^4*z^2 - y^6*z^2 - x^4*z^4 - 2*x^2*y^2*z^4 - y^4*z^4 - x^2*z^6 - y^2*z^6 - x^4*y^2 - x^2*y^4 - x^4*z^2 - 3*x^2*y^2*z^2 - y^4*z^2 - x^2*z^4 - y^2*z^4 + x^4 + y^4 + z^4 + x^2 + y^2 + z^2 - 1)/(x^8*y^8*z^8 - x^8*y^8*z^6 - x^8*y^6*z^8 - x^6*y^8*z^8 - x^8*y^8*z^4 + x^8*y^6*z^6 + x^6*y^8*z^6 - x^8*y^4*z^8 + x^6*y^6*z^8 - x^4*y^8*z^8 + x^8*y^8*z^2 + x^8*y^6*z^4 + x^6*y^8*z^4 + x^8*y^4*z^6 - x^6*y^6*z^6 + x^4*y^8*z^6 + x^8*y^2*z^8 + x^6*y^4*z^8 + x^4*y^6*z^8 + x^2*y^8*z^8 - x^8*y^6*z^2 - x^6*y^8*z^2 + x^8*y^4*z^4 - x^6*y^6*z^4 + x^4*y^8*z^4 - x^8*y^2*z^6 - x^6*y^4*z^6 - x^4*y^6*z^6 - x^2*y^8*z^6 - x^6*y^2*z^8 + x^4*y^4*z^8 - x^2*y^6*z^8 - x^8*y^4*z^2 + x^6*y^6*z^2 - x^4*y^8*z^2 - x^8*y^2*z^4 - x^6*y^4*z^4 - x^4*y^6*z^4 - x^2*y^8*z^4 + x^6*y^2*z^6 - x^4*y^4*z^6 + x^2*y^6*z^6 - x^4*y^2*z^8 - x^2*y^4*z^8 + x^8*y^2*z^2 + x^6*y^4*z^2 + x^4*y^6*z^2 + x^2*y^8*z^2 + x^6*y^2*z^4 - x^4*y^4*z^4 + x^2*y^6*z^4 + x^4*y^2*z^6 + x^2*y^4*z^6 + x^2*y^2*z^8 - x^6*y^2*z^2 + x^4*y^4*z^2 - x^2*y^6*z^2 + x^4*y^2*z^4 + x^2*y^4*z^4 - x^2*y^2*z^6 - x^4*y^2*z^2 - x^2*y^4*z^2 - x^2*y^2*z^4 + x^2*y^2*z^2))*q^2 + O(q^3)

In [14]:
_ == v000 # check latest output == the vertex

True

In [15]:
# Restrict to the Calabi-Yau torus xyz = 1. In this setting, the self-duality
# of the deformation theory ensures all equivariant weights vanish.
v000(x=x, y=y, z=1/(x*y))

1 + q + 3*q^2 + O(q^3)

In [16]:
############################################################
# Let's look at some other features: 
# - using custom variables;
# - descendent insertions;
# - using cohomology instead of K-theory;
# - reduced DT series (using .series instead of .series_unreduced)

R.<q1,q2> = PolynomialRing(QQ)
QR.<Q> = PowerSeriesRing(R.fraction_field())

V100coh = BareVertex([1], [], [], ct=Cohomology)
ch5 = lambda f: Cohomology.chern_character(5, f)
pretty( V100coh.series(3, x=q1, y=q2, z=-q1-q2, q=Q, descendant=ch5) )

1/24*(q1^2 + q1*q2 + q2^2)*(q1 + q2)*q1*q2 + (1/24*(17*q1^2 + 29*q1*q2 + 29*q2^2)*(q1 + q2)*q1*q2)*Q + (1/24*(101*q1^2 + 185*q1*q2 + 185*q2^2)*(q1 + q2)*q1*q2)*Q^2 + O(Q^3)

In [17]:
# Check the K-theoretic DT/PT correspondence
BareVertex([2], [1], [1]).series(3) == BareVertexPT([2], [1], [1]).series(3)

True

In [18]:
# PT configurations are much more combinatorially involved than 
# DT configurations
PTConfigurations([2], [3], [4,2]).random_element_with_num_boxes(25).plot()

Graphics3d Object